# YAM 328 Makine Öğrenmesine Giriş
# Proje ödevi
****

**UCI-Energy Efficiency dataset üzerinde ML ile Isıtma ve Soğutma yükünü tahmin etme ve doğruluk testi.**


# Gerekli Kütüphaneleri içe aktarıyoruz.

In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# **-Veri ön İşleme-**
# Veri setini seçiyoruz
# data.info() ile kayıp veri olup olmadığını görüyoruz
****

In [ ]:
data = pd.read_csv('../input/energy-efficiency-data-set/energy_efficiency_data.csv')
data.info()
data

# **-Özellik Seçimi-**
# A-)Normalizasyon uyguluyoruz
****

In [ ]:
def normalization(data,col):
    data[col]=[(i-data[col].min())/(data[col].max()-data[col].min())
              for i in data[col]]
columns_to_normalization=['Relative_Compactness','Surface_Area','Wall_Area',
                          'Roof_Area','Overall_Height','Orientation','Glazing_Area',
                          'Glazing_Area_Distribution','Heating_Load','Cooling_Load']
for column in columns_to_normalization:
    normalization(data,column)
data    

# B-) Özellik seçimi için
**özelliklerin ve sonuçların histogram çıktıları**
****

In [ ]:
num_list = list(data.columns)

fig = plt.figure(figsize=(10,30))

for i in range(len(num_list)):
    plt.subplot(15,2,i+1)
    plt.title(num_list[i])
    plt.hist(data[num_list[i]],color='blue',alpha=0.5)

plt.tight_layout()

# B-) Özellik seçimi için
**korelasyona bakarak veriler arasındaki ilişkiyi değerlendirebiliriz**
****

In [ ]:
plt.figure(figsize = (10,8))
sns.heatmap(data.corr(),annot=True, cbar=False, cmap='Blues', fmt='.1f')

# Hedef verilerimiz heating_load ve cooling_load
****

In [ ]:
sns.histplot(data['Cooling_Load'],color="blue",kde=True, stat="density", linewidth=0)
sns.histplot(data['Heating_Load'],color="red",kde=True, stat="density", linewidth=0)
plt.legend(['Cooling Load','Heating Load'])
plt.xlabel('Load')
plt.show()

# Modelleme

> ****Modelleme için hedef özelliklerimizin olduğu veriyi kopyalıyoruz****

In [ ]:
df = data.copy()
X = df[['Relative_Compactness', 'Surface_Area', 'Wall_Area', 'Roof_Area',
       'Overall_Height', 'Orientation', 'Glazing_Area','Glazing_Area_Distribution']]
y_h = df[['Heating_Load']]
y_c = df[['Cooling_Load']]

# normalizasyon uygulanmış veriyi kullanma


In [ ]:
mmx_scaler = MinMaxScaler()
stand_scalar = StandardScaler()
X_normalized = mmx_scaler.fit_transform(X)
X_standarized = stand_scalar.fit_transform(X)

# normalize edilmiş veri setini eğitim ve test verisi olarak bölümleme

In [ ]:
X_train, X_test, yh_train, yh_test, yc_train, yc_test = train_test_split(X_normalized,y_h,y_c, 
                                                                         test_size=0.33, random_state = 42)

# Makine Öğrenmesi Modelleri

In [ ]:
Acc = pd.DataFrame(index=None, 
                   columns=['model','train_Heating','test_Heating','train_Cooling','test_Cooling'])
regressors =[ ['DecisionTreeRegressor',DecisionTreeRegressor()],
              ['KNeighborsRegressor', KNeighborsRegressor()],
              ['LinearRegression',LinearRegression()]]

# Makine öğrenme modellerinin doğruluk oranı

In [ ]:
for mod in regressors:
    name = mod[0]
    model = mod[1]
    
    model.fit(X_train,yh_train)
    actr1 = r2_score(yh_train, model.predict(X_train))
    acte1 = r2_score(yh_test, model.predict(X_test))
    yh_pred=model.predict(X_test)
    
    model.fit(X_train,yc_train)
    actr2 = r2_score(yc_train, model.predict(X_train))
    acte2 = r2_score(yc_test, model.predict(X_test))
    yc_pred=model.predict(X_test)

    Acc = Acc.append(pd.Series({'model':name, 'train_Heating':actr1,'test_Heating':acte1,'train_Cooling':actr2,'test_Cooling':acte2}),ignore_index=True)
    
 

Acc.sort_values(by='test_Cooling')



In [ ]:
x_ax = range(len(yc_test))
plt.figure(figsize=(20,10))
plt.subplot(2,1,1)
plt.plot(x_ax, yh_test, label="Actual Heating")
plt.plot(x_ax, yh_pred, label="Predicted Heating")
plt.title("Heating test and predicted data")
plt.xlabel('X-axis')
plt.ylabel('Heating load (kW)')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)

plt.subplot(2,1,2)
plt.plot(x_ax, yc_test, label="Actual Cooling")
plt.plot(x_ax, yc_pred, label="Predicted Cooling")
plt.title("Coolong test and predicted data")
plt.xlabel('X-axis')
plt.ylabel('Cooling load (kW)')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)

plt.show()

# En yüksek korelasyon değerli olan yöntem DecisionTreeRegressor() çıktı. 
> **aşağıda ağacın yaprak, node ve dallanma sayıları yazıyor.**

> **criterion: mean squared error (MSE) and mean absolute error (MAE)**

In [ ]:
from sklearn.model_selection import GridSearchCV
DTR = DecisionTreeRegressor()
param_grid = {"criterion": ["mse", "mae"],"min_samples_split": [14, 15, 16, 17],
              "max_depth": [5, 6, 7],"min_samples_leaf": [4, 5, 6],"max_leaf_nodes": [29, 30, 31, 32],}

grid_cv_DTR = GridSearchCV(DTR, param_grid, cv=5)

grid_cv_DTR.fit(X_train,yc_train)
print("R-Squared::{}".format(grid_cv_DTR.best_score_))
print("Best Hyperparameters::\n{}".format(grid_cv_DTR.best_params_))

In [ ]:
DTR = DecisionTreeRegressor(criterion= 'mse', max_depth= 6, max_leaf_nodes= 30, min_samples_leaf= 5, min_samples_split= 17)

DTR.fit(X_train,yh_train)
print("R-Squared on train dataset={}".format(DTR.score(X_test,yh_test)))

DTR.fit(X_train,yc_train)   
print("R-Squaredon test dataset={}".format(DTR.score(X_test,yc_test)))